# Introduction
In this small project we will take a look at Seattle weather dataset from Kaggle to extract important features and use them to test functionality of Recurrent Neural Network.
<br><br>
#### Main objective
Build a model using PyTorch library that predicts temperature for the next day.
<br><br>
#### Process includes:
1. **Data** <br>
&ensp;1.1 Overview <br>
&ensp;1.2 Anomalies <br>
&ensp;1.3 Visualization <br>
&ensp;1.4 Preparing data for model <br>
2. **Preparing Dataset** <br>
&ensp;2.1 Load data<br>
&ensp;2.2 Feature scaling <br>
&ensp;2.3 Train / test split <br>
&ensp;2.4 Data loader <br>
3. **Building Model** <br>
&ensp;3.1 Class RNN<br>
&ensp;3.2 Training <br>
&ensp;3.3 Testing <br>
&ensp;3.4 Second model <br>
&ensp;3.5a Cross-validation for single feature <br>
&ensp;3.5b Cross-validation for multiple features <br>
&ensp;3.6 Comparison <br>
4. **Conclusion** <br>

#### Setup
Matplotlib color scheme.

In [ ]:
import matplotlib.pyplot as plt

WHITE_MID = '#b5b5b5'
GREY_DARK = '#141414'

plt.rcParams['figure.facecolor'] = GREY_DARK
plt.rcParams['text.color'] = WHITE_MID
plt.rcParams['axes.facecolor'] = GREY_DARK
plt.rcParams['axes.edgecolor'] = WHITE_MID
plt.rcParams['axes.labelcolor'] = WHITE_MID
plt.rcParams['axes.titlecolor'] = WHITE_MID

plt.rcParams['grid.color'] = WHITE_MID
plt.rcParams['grid.linestyle'] = '--'
plt.rcParams['grid.linewidth'] = 0.5
plt.rcParams['axes.grid'] = True

plt.rcParams['axes.linewidth'] = 1
plt.rcParams['xtick.color'] = WHITE_MID
plt.rcParams['ytick.color'] = WHITE_MID
plt.rcParams['legend.edgecolor'] = WHITE_MID
plt.rcParams['legend.labelcolor'] = WHITE_MID

Random seeds, cuda and warnings.

In [ ]:
import numpy as np
import torch
import random
import warnings

warnings.filterwarnings("ignore")

seed = 42

random.seed(seed)
np.random.seed(seed)

# the model is not reproducible probably due to training on GPU (CUDA)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

# II - Preparing Dataset
#### 2.1 - Load data

In [ ]:
import pandas as pd

df = pd.read_csv('../datasets/weather-processed.csv')
df

#### 2.2 - Feature scaling
Using _**MinMaxScaler**_ from sklearn.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
features = ['precipitation', 'temp_avg', 'wind']

df.loc[:, features] = scaler.fit_transform(df[features]) # applying scaled features to numeric features

df.describe()

#### 2.3 - Train / test split

In [ ]:
train_size = int(len(df) * 0.7) + 1
test_size = int(len(df) * 0.3)

print(
    f"{f"train size":>12}: {train_size}",
    f"{f"test size":>12}: {test_size}",
    len(df) == train_size + test_size,
    sep='\n'
)

In [ ]:
df_train = df[:train_size]
df_test = df[train_size:]

We're going to pass sequenced data with length of n days per sequence. <br>
Knowing that, we have to make sure that our datasets are divisible by n. <br><br>
With _sequence_len_ parameter we can adjust length of sequence in days (1 day, week, month) and test how it affects models predictions. <br>
Despite this there's no enough data to test this model on longer sequences like month, so I chose one week.

In [ ]:
sequence_len = 7

df_train = df_train[:(len(df_train) // sequence_len) * sequence_len]
df_test = df_test[:(len(df_test) // sequence_len) * sequence_len]

print(
    len(df_train),
    len(df_test), sep='\n'
)

Convert each data split to numpy, so we're able to pass it to torch.

In [ ]:
df_train_torch = df_train.to_numpy()
df_train_torch = torch.tensor(df_train_torch)

df_test_torch = df_test.to_numpy()
df_test_torch = torch.tensor(df_test_torch)
df_test_torch[:7]

#### 2.4 - Data loader
To predict temperature based on features from past n days, <br>
we have to reshape our data split sets to return specific sequences. <br><br>
Each sequence containing data from 7 days, where: <br>
X - temp_avg from first n days <br>
y - same feature, but we take just the last day

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

def sequence_dataset(dataset, batch_size, extra_features=False):
    
    x, y = [], []
    
    for i in range(0, len(dataset) - sequence_len, sequence_len):
        
        # if extra_features = True, then x = [precipitation, wind, temp_avg], y = temp_avg
        # if extra_features = False, then x = temp_avg, y = temp_avg
        if extra_features:
            week = dataset[i: i + sequence_len, :] # take precipitation, wind and temperature (:)
            week_input = week[:sequence_len-1] # trim to n days
            week_target = week[-1, 2] # take last day
        else:
            week = dataset[i: i + sequence_len, 2] # take just the temperature (2)
            week_input = week[:sequence_len-1] # trim to n days
            week_target = week[-1] # take last day
        
        x.append(week_input)
        y.append(week_target)
        
    # covert to tensor dataset
    tensor_dataset = TensorDataset(torch.tensor(np.asarray(x)),
                                   torch.tensor(np.asarray(y)))
    
    # covert to data_loader
    data_loader = DataLoader(tensor_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

    return data_loader

In [ ]:
batch_size = 7
train_loader = sequence_dataset(df_train_torch, batch_size=batch_size)
test_loader = sequence_dataset(df_test_torch, batch_size=batch_size)

In [ ]:
for batch in train_loader:
    input, target = batch
    print(input[0], target[0], sep='\n')
    break

# III - Building Model
#### 3.1 - Class RNN
With PyTorch we are able to build simple RNN with Long Short-Term Memory layers by simply attaching _nn.LSTM()_ with specific factors in it. <br>
The __init__ function first initializes the LSTM and fully connected layers. <br>
__*forward()*__ passes our input x to the rnn network then fully connects it to finally return last value from the output.

In [ ]:
from torch import nn

class WeatherRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size=3, n_layers=2, dropout=0):
        super(WeatherRNN, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers=n_layers, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :]) # take the last hidden
        return out

#### 3.2 - Training
So we take our train_loader and pass it to our training function. <br>
Training function iterates over batches for n_epoch times and passes sequences from these batches to our model <br>
that calculates loss by comparing calculated outputs with actual targets using Mean Squared Error function.

In [ ]:
import torch.optim as optim
import copy

def train_model(model, device, train_loader, n_epochs=1, lr=0.001, extra_features=False, verbose=1):
    
    model.to(device)
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    loss_history = []
    best_loss = float('inf')
    best_epoch = 0
    best_weights = copy.deepcopy(model.state_dict()) # init weights
    
    for epoch in range(1, n_epochs + 1):
        model.train()
        epoch_loss = 0
        avg_loss = 0
        
        for i, batch in enumerate(train_loader):
            inputs, targets = batch
            inputs, targets = (inputs.to(device).float(),
                               targets.to(device).float())
            
            if not extra_features:
                inputs = inputs.unsqueeze(-1)
            
            optimizer.zero_grad() # clean gradients
            outputs = model(inputs) # calculate outputs
            
            loss = criterion(outputs, targets.unsqueeze(1)) # add dim
            loss.backward() # backpropagation
            optimizer.step() # update weights

            epoch_loss += loss.item()
            avg_loss = epoch_loss / len(train_loader)
        
        # check if current epoch is the best one
        if avg_loss < best_loss:
            best_loss = avg_loss
            best_weights = copy.deepcopy(model.state_dict())
            best_epoch = epoch - 1
            
        if verbose:
            if epoch % 10 == 0 or epoch == 1:
                print(f"Epoch {f"{epoch}":>02} / {n_epochs} finished | {epoch_loss/len(train_loader):.4f}")
                
        loss_history.append(epoch_loss)
        
    model.load_state_dict(best_weights)
    
    return loss_history, best_epoch

In [ ]:
input_size = 1
output_size = 1
hidden_size = 64
n_layers = 1
dropout = 0
learning_rate = 0.0001
n_epochs = 50

weather_rnn = WeatherRNN(input_size=input_size, hidden_size=hidden_size, n_layers=n_layers, output_size=output_size, dropout=dropout)

loss_history, best_epoch = train_model(weather_rnn, device=device, train_loader=train_loader, n_epochs=n_epochs, lr=learning_rate)

In [ ]:
import seaborn as sb

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(20, 8))
axes = axes.flatten()

plt.suptitle('Weather RNN - Loss')

sb.lineplot(loss_history, ax=axes[0])
axes[0].set_xticks(range(0, n_epochs + 1, 2))
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].axvline(x=best_epoch, color='g', linestyle=':')

loss_history_rescaled = np.zeros((len(loss_history), 3))
loss_history_rescaled[:, 2] = np.array(loss_history)
loss_history_rescaled = scaler.inverse_transform(loss_history_rescaled)[:, 2]

sb.lineplot(loss_history_rescaled, ax=axes[1])
axes[1].set_xticks(range(0, n_epochs + 1, 2))
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss (Temperature)')
axes[1].set_ylim((0, 5))
axes[1].axvline(x=best_epoch, color='g', linestyle=':')

plt.tight_layout()

#### 3.3 - Testing
Similar to training function. This time we run the model with no gradients calculations <br>
and iterate through batches from test_loader. <br>
Then return y true and y predicted values to compare where the model makes most mistakes.

In [ ]:
# rescaling score data back to temperature format.
def to_degrees(feature):
    feature_deg = np.zeros((len(feature), 3)) # because our scaler remember what did it scale before, we need to create data with same dimensions
    feature_deg[:, 2] = np.sqrt(feature) # square root, because we use Mean Square Error in our models
    feature_deg = scaler.inverse_transform(feature_deg)
    return feature_deg[:, 2]

In [ ]:
def test_model(model, device, test_loader, extra_features=False):
    total_loss = 0
    criterion = nn.MSELoss()
    y_true, y_pred = [], []

    with torch.no_grad():
        for batch in test_loader:
            inputs, targets = batch
            inputs, targets = (inputs.to(device).float(),
                               targets.to(device).float())
            
            if not extra_features:
                inputs = inputs.unsqueeze(-1)

            outputs = model(inputs) # get outputs

            loss = criterion(outputs, targets.unsqueeze(1)) # add dim
            total_loss += loss.item()

            y_true.extend(targets.cpu().numpy())
            y_pred.extend(outputs.cpu().numpy())

    avg_loss = total_loss / len(test_loader)
    print(f"Test MSE Loss: {avg_loss:.4f}")

    return y_true, y_pred, avg_loss

In [ ]:
import seaborn as sb

y_true, y_pred, mse = test_model(weather_rnn, device=device, test_loader=test_loader)

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(24, 10))
axes = axes.flatten()

temp_avg_true = np.array(y_true).flatten()
temp_avg_pred = np.array(y_pred).flatten()

x = range(len(temp_avg_true))
axes[0].vlines(x=x, ymin=temp_avg_pred, ymax=temp_avg_true, color='red', linewidth=1, label='error')
axes[0].plot(x, y_pred, marker='+', linestyle='None', color='white', label='pred')

sb.lineplot(x=range(len(temp_avg_true)), y=temp_avg_true, color='red', ax=axes[1], label='true')
sb.lineplot(x=range(len(temp_avg_pred)), y=temp_avg_pred, color='white', ax=axes[1], label='pred')

axes[0].set_title('Predictions Errors')
axes[0].legend()

axes[1].set_title('True and Predicted overlap')

plt.suptitle('Predictions Summary', fontweight='bold')
plt.tight_layout()

true_deg = to_degrees(temp_avg_true)
pred_deg = to_degrees(temp_avg_pred)

error = abs(true_deg - pred_deg)

print(
    "\nError in degrees:\n"
    f"{"mean":<6}: {error.mean():4f}",
    f"{"max":<6}: {error.max():4f}",
    f"{"min":<6}: {error.min():4f}",
    f"{"range":<6}: {(true_deg.max() - true_deg.min()):4f}",
    sep='\n'
)

#### 3.4 - Second model
Now we use 3 features (precipitation, wind, temp_avg) to predict just a temp_avg.


In [ ]:
train_loader_2 = sequence_dataset(df_train_torch, batch_size=batch_size, extra_features=True)
test_loader_2 = sequence_dataset(df_test_torch, batch_size=batch_size, extra_features=True)

In [ ]:
i = 0
for batch in train_loader_2:
    if i == 0:
        input, target = batch
        # print(input, target)
        print(input.size())
        print(target.size())
        break
    i += 1

In [ ]:
weather_rnn_2 = WeatherRNN(input_size=3,
                           hidden_size=hidden_size,
                           n_layers=n_layers,
                           output_size=output_size,
                           dropout=dropout)

loss_history_2, best_epoch_2 = train_model(weather_rnn_2, device=device, train_loader=train_loader_2, n_epochs=n_epochs, lr=learning_rate, extra_features=True)

In [ ]:
y_true_2, y_pred_2, mse_2 = test_model(weather_rnn_2, device=device, test_loader=test_loader_2, extra_features=True)

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(24, 10))
axes = axes.flatten()

temp_avg_true_2 = np.array(y_true_2).flatten()
temp_avg_pred_2 = np.array(y_pred_2).flatten()

x = range(len(temp_avg_true_2))
axes[0].vlines(x=x, ymin=temp_avg_pred_2, ymax=temp_avg_true_2, color='red', linewidth=1, label='error')
axes[0].plot(x, y_pred_2, marker='+', linestyle='None', color='white', label='pred')

sb.lineplot(x=range(len(temp_avg_true_2)), y=temp_avg_true_2, color='red', ax=axes[1], label='true')
sb.lineplot(x=range(len(temp_avg_pred_2)), y=temp_avg_pred_2, color='white', ax=axes[1], label='pred')

axes[0].set_title('Prediction errors')
axes[0].legend()

axes[1].set_title('True and Predicted values overlap')

plt.suptitle('Predictions Summary', fontweight='bold')
plt.tight_layout()

true_deg_2 = to_degrees(temp_avg_true_2)
pred_deg_2 = to_degrees(temp_avg_pred_2)

error_2 = abs(true_deg_2 - pred_deg_2)

print(
    "\nError in degrees:\n"
    f"{"mean":<6}: {error_2.mean():4f}",
    f"{"max":<6}: {error_2.max():4f}",
    f"{"min":<6}: {error_2.min():4f}",
    f"{"range":<6}: {(true_deg_2.max() - true_deg_2.min()):4f}",
    sep='\n'
)

#### 3.5a - Cross-validation for single feature
To test the model on the whole dataset without overfitting we can commit _**Cross-Validation**_.

In [ ]:
# n_folds is number of chunks we split our dataset for
n_folds = 5
df = df.iloc[: (len(df) // n_folds) * n_folds]
step = len(df) // n_folds

df_splits = []

for i in range(0, len(df), step):
    test_chunk = df.iloc[i: i+step] # select certain chunk as test_chunk
    train_chunk = df.drop(df[i: i+step].index) # and then remove the same chunk from train_chunk
    df_splits.append((train_chunk, test_chunk))

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

cross_val_errors_1 = [] # a list that contains all errors for single feature model for each fold

# cross-validation
for i, split in enumerate(df_splits):
    train, test = split
    
    # train loader
    train_torch = train.to_numpy()
    train_torch = torch.tensor(train_torch)
    train_loader = sequence_dataset(train_torch, batch_size=batch_size, extra_features=False)
    
    # test loader
    test_torch = test.to_numpy()
    test_torch = torch.tensor(test_torch)
    test_loader = sequence_dataset(test_torch, batch_size=batch_size, extra_features=False)
    
    nth_model = WeatherRNN(input_size=1, hidden_size=hidden_size, n_layers=n_layers, output_size=output_size, dropout=dropout) # define model
    
    print(f'Fold {i+1} - ', end='')
    train_model(nth_model, device=device, train_loader=train_loader, n_epochs=n_epochs, lr=learning_rate, extra_features=False, verbose=0) # train model
    
    y_true, y_pred, mse = test_model(nth_model, device=device, test_loader=test_loader, extra_features=False) # test model
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    error = abs(y_true - y_pred) # absolute value of prediction errors

    cross_val_errors_1.append(error)
    
print(f"MSE Loss for {n_folds} folds: {mse:4f}")
print(f"Error for {n_folds} folds: {np.mean(cross_val_errors_1):4f}")
print(f"Global MAE for {n_folds} folds: {mean_absolute_error():4f}")

#### 3.5b Cross-validation for multiple features
To test the model with multiple features in cross-validation we simply change _**extra_features**_ boolean value to True in: <br>
_train_loader_, _test_loader_, _train_model_ and _test_model_. <br>
This will pass all the features (precipitation, wind, temp_avg) to all functions above.

In [ ]:
np.mean([0.0070, 0.0082, 0.0082, 0.0099, 0.0100])

In [ ]:
cross_val_errors_2 = [] # a list that contains all errors for multiple feature model for each fold

# cross-validation
for i, split in enumerate(df_splits):
    train, test = split

    # train loader
    train_torch = train.to_numpy()
    train_torch = torch.tensor(train_torch)
    train_loader = sequence_dataset(train_torch, batch_size=batch_size, extra_features=True)

    # test loader
    test_torch = test.to_numpy()
    test_torch = torch.tensor(test_torch)
    test_loader = sequence_dataset(test_torch, batch_size=batch_size, extra_features=True)

    nth_model = WeatherRNN(input_size=3, hidden_size=hidden_size, n_layers=n_layers, output_size=output_size, dropout=dropout) # define model

    print(f'Fold {i+1}:')
    train_model(nth_model, device=device, train_loader=train_loader, n_epochs=n_epochs, lr=learning_rate, extra_features=True) # train model

    y_true, y_pred = test_model(nth_model, device=device, test_loader=test_loader, extra_features=True) # test model
    y_true, y_pred = np.array(y_true), np.array(y_pred)

    error = abs(y_true - y_pred) # absolute value of prediction errors
    print('')

    cross_val_errors_2.append(error)

print(f"Average error for {n_folds} folds: {np.mean(cross_val_errors_2):4f}")

#### 3.6 - Comparison
Now we create one big DataFrame that contains all information that cross-validation produced.

In [ ]:
records = []

# n_features stands for features used to predict temperature (e.g. n_features=3: precip, wind, temp -> temp)
for n_features, data in zip([1, 3], [cross_val_errors_1, cross_val_errors_2]):
    data = np.stack(data)
    for fold in range(n_folds):
        fold_data = data[fold] # take certain data by fold idx
        records.append({
            'n_features': n_features,
            'fold': str(fold),
            'mean': fold_data.mean(),
            'min': fold_data.min(),
            'max': fold_data.max(),
            'errors': fold_data[fold]
        })

cross_val_df = pd.DataFrame(records)

In [ ]:
cross_val_df['mean'].max()

In [ ]:
cross_val_df['min'].min()

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(20, 20))
axes = axes.flatten()

model_1 = cross_val_df.n_features == 1
model_3 = cross_val_df.n_features == 3

sb.barplot(data=cross_val_df, x='mean', y='fold', hue='n_features', palette="muted", linewidth=0, ax=axes[0])
sb.barplot(data=cross_val_df, x='min', y='fold', hue='n_features', palette="muted", linewidth=0, ax=axes[1])
sb.barplot(data=cross_val_df, x='max', y='fold', hue='n_features', palette="muted", linewidth=0, ax=axes[2])

axes[0].set_xlim(cross_val_df['mean'].min() - .005, cross_val_df['mean'].max() + .005)
axes[1].set_xlim(cross_val_df['min'].min() - .005, cross_val_df['min'].max() + .005)
axes[2].set_xlim(cross_val_df['max'].min() - .005, cross_val_df['max'].max() + .005)

plt.show()

# V - Conclusion
The chaos in the _precipitation_ and _wind_ series doesn't seem to provide any valuable information that helps the model predict the average temperature. <br>
Despite this fact the model did decent job with predicting average temperature, both for weekly and daily sequences. <br><br>
Overall I think this project gives quiet simple fundamentals to understand the functionality of Recurrent Neural Networks.
#### Thank you for reading my project, I hope you enjoyed the process :]
_- Gracjan Pawłowski, 2025_